In [1]:
import torch
from torch import nn

from torch.optim import lr_scheduler

from torch.utils.data import random_split,Dataset,DataLoader

import torch.nn.functional as F
import torch.nn.init as init

from momentumnet import MomentumNet
from momentumnet import transform_to_momentumnet

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from joblib import Parallel, delayed
from joblib.externals.loky.backend.context import get_context

import time
import copy
import random
import pickle
import tarfile

import time
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# Dataset Class

In [2]:
class MyDataset(Dataset):
    def __init__(self,gene_matrix,cell_type):
        
        self.gene_matrix = torch.from_numpy(gene_matrix).float()
        self.cell_type = torch.from_numpy(cell_type).squeeze(1)
    

    def __len__(self):
        
        return self.gene_matrix.shape[0]

    def __getitem__(self,idx):
        
        data = (self.gene_matrix[idx],self.cell_type[idx])
        
        return data

# Create PBMC DataLoader

In [3]:
def create_pbmc1_loader(pbmc_path,cell_type_path,batch_size=128):

    pbmc = pd.read_csv(pbmc_path,header=None)
    cell_type = pd.read_csv(cell_type_path,index_col = 0)
    
    full_dataset = MyDataset(pbmc.values,cell_type.values)

    #Random split(0.7,0.3)
    train_size = int(0.7 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

    # Define DataLoaders
    # use 'loky' to work with joblib
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,   batch_size=batch_size, shuffle=False)
    
    return train_loader, test_loader


# MLP

## Initial

In [4]:
def _weights_init(m):
    classname = m.__class__.__name__
    #print(classname)
    if isinstance(m, nn.Linear):
        init.normal(m.weight)

## MLP Model

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        
        self.lr1 = nn.Linear(3500,1000)
        self.lr2 = nn.Linear(1000,128)
        self.lr3 = nn.Linear(128,64)
        
        self.lr = nn.Linear(64,9)
        
        self.apply(_weights_init)
        
    def forward(self,x):
        x = F.relu(self.lr1(x))
        x = self.lr2(x)
        x = F.relu(self.lr3(x))
        
        output = self.lr(x)
        
        return output

# SGD_Train

In [6]:
def SGD_training(network, SGD_steps, lr, momentum, data_loader):

    network.to(device)
    network.train()

    # Create optimizer and criterion
    criterion = nn.CrossEntropyLoss(reduction='mean')
    
    optimizer = torch.optim.SGD(network.parameters(), lr=lr, momentum=momentum, nesterov=True, weight_decay=0.0001)
    
    total_step = len(data_loader)
    
    for s in range(0, SGD_steps):
        total_loss = 0
        for i, (genes, types) in enumerate(data_loader): 
            
            genes = genes.to(device)
            types = types.to(device)
            
            # Forward pass
            outputs = network(genes) 
            
            loss = criterion(outputs, types)
            total_loss += loss.item()
            # Backward and optimize
            optimizer.zero_grad()
            
            loss.backward()
            
            #Gradient Value Clipping
            nn.utils.clip_grad_value_(network.parameters(), clip_value=1.0)
            
            optimizer.step()
            
            del loss, outputs
            
            #if (i+1) % 100 == 0:
        total_loss = total_loss / len(data_loader.sampler)
        print ("Epoch [{}/{}], Loss: {}".format(s+1, SGD_steps, total_loss))
        
            
            
    
    # move network back to cpu and return
    network.cpu()
    
    return network

# GA

In [7]:
def crossover_and_mutation(parents, sigma=0.01):

    
    base_sd = parents[0].state_dict()
    keys = base_sd                    # use all layers to be affected
    
    # Sum of the weights of the parent
    for i in range(1, len(parents)):
        parent_sd = parents[i].state_dict()
        for key in keys:
            base_sd[key] = base_sd[key] + parent_sd[key]
            
    
    # Average and add mutation
    num_parents = len(parents)
    
    for key in keys:
        
        tensor_size = base_sd[key].size()
        random_tensor = torch.normal(mean=0.0, std=sigma, size=tensor_size)
        
        base_sd[key] = (base_sd[key] / num_parents) + random_tensor
    
    # create offspring
    offspring = MLP()
    
    offspring.load_state_dict(base_sd)
    
    return offspring
    

def create_offspring(population, fitness, rho, sigma):

    
    # Perform selection
    parents = random.choices(population, weights=fitness, k=rho) 
    
    # Perform crossover and mutation
    offspring = crossover_and_mutation(parents, sigma)
    
    
    return offspring

def sigmoid(x):
    
    return 1/(1 + np.exp(-x))


def GA_training(population, pop_size, offspring_size, elitist_level, rho, sigma, data_loader):
    
    #Calculate fitness of trained population

    fitness = [calc_loss(population[i], data_loader) for i in range(pop_size)]
    
    print(f"--- -- Finished fitness evaluation, length: {len(fitness)}")
    
    #Create offspring population
    fitness_weighted = [sigmoid(-f) for f in fitness]   # take inverse of loss so lower losses get higher fitness-values
    offspring_population = [create_offspring(population, fitness_weighted, rho, sigma) for i in range(offspring_size)]
    print("--- -- Finished creating offspring population")
    
    #Evaluate fitness of offsprings 
    
    offspring_fitness = [calc_loss(offspring_population[i], data_loader) for i in range(offspring_size)]
    print("--- -- Finished evaluating fitness of offspring population")
    
    # Combine fitness and population lists
    
    combined_fitness = fitness + offspring_fitness
    combined_population = population + offspring_population
    
    # sort and select population by their fitness values
    
    sorted_population = [pop for _, pop in sorted(zip(combined_fitness, combined_population), key=lambda pair: pair[0])]
    sorted_fitness = [loss for loss, _ in sorted(zip(combined_fitness, combined_population), key=lambda pair: pair[0])]
    
    m = int(pop_size * elitist_level)
    new_population = sorted_population[0:m]
    
    # Fill up rest of population
    difference = pop_size - m
    remaining_population = list(set(sorted_population) - set(new_population))
    filler_population = random.sample(remaining_population, difference)
    
    # assemble new population and return
    new_population = new_population + filler_population
    
    return new_population, sorted_fitness

In [8]:
def calc_loss(network, data_loader):
    
    network.to(device)
    criterion = nn.CrossEntropyLoss(reduction='mean')
    total_loss = 0.0
    
    for i, (genes, types) in enumerate(data_loader):
        
        genes = genes.to(device)
        types = types.to(device)
        
        # Forward pass
        outputs = network(genes)
        loss = criterion(outputs, types)
        
        total_loss += loss
        
        del loss, outputs
        
    network.cpu()
    
    return float(total_loss/len(data_loader.sampler))

# GA_Neural

In [9]:
def GA_Neural_train(population,
                    pop_size,
                    max_generations, 
                    SGD_steps, GA_steps, 
                    offspring_size, elitist_level, rho,
                    learning_rate,
                    train_loader):
    
    print(f"Starting with population of size: {pop_size}")
    
    
    for k in range(max_generations):
        print(f"Currently in generation {k+1}")
        
        #SGD
        print(f"--- Starting SGD")
        
        # Sequential version
        population = [SGD_training(population[i], SGD_steps, learning_rate, 0.9, train_loader) for i in range(pop_size)]
        
        print(f"--- Finished SGD")
         
        # GA
        print(f"--- Starting GA")
        GA_start = time.time()
        sorted_fitness = []          # store the sorted fitness values to maybe use in data collection
        for i in range(0, GA_steps):
            
            sigma = 0.01 / (k+1)
            population, sorted_fitness = GA_training(population, pop_size, offspring_size, elitist_level, rho, sigma, train_loader)
        
        GA_end = time.time()
        print(f"--- Finished GA,Time:{(GA_start-GA_end)*1000}ms")
        
        
    print(f"Finished training process")
    return population

# Start training process
We have now defined the whole training algorithm. The next step is to actually perform training.

In [10]:
# Hyperparameters
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
batch_size = 128
pop_size = 2
max_generations = 10
SGD_steps = 10
GA_steps = 1
offspring_size = 2
elitist_level = 0.6
rho = 2
learning_rate = 1e-5
pbmc_1 = "data/pbmc_1_pca.csv"
cell_type_pbmc1 = "data/cell_type_pbmc1.csv"
pbmc_2 = "data/pbmc_2_pca.csv"
cell_type_pbmc2 = "data/cell_type_pbmc2.csv"

In [11]:
train_loader, test_loader = create_pbmc1_loader(pbmc_1,cell_type_pbmc1,batch_size = batch_size)

In [12]:
# Create population and start training process
population = [MLP() for i in range(pop_size)]

# Train Process

In [13]:
Train_start = time.time()
trained_population = GA_Neural_train(population=population,
                                    pop_size = pop_size,
                                    max_generations=max_generations,
                                    SGD_steps=SGD_steps,GA_steps=GA_steps,
                                    offspring_size=offspring_size,elitist_level=elitist_level,rho=rho,
                                    learning_rate=learning_rate,
                                    train_loader=train_loader)
Train_end = time.time()
print(f"All Time:{(Train_start-Train_end)*1000}ms")

Starting with population of size: 2
Currently in generation 1
--- Starting SGD
Epoch [1/10], Loss: 2035.1902551020407
Epoch [2/10], Loss: 1677.2460331632653
Epoch [3/10], Loss: 1352.1584725765306
Epoch [4/10], Loss: 1175.195306122449
Epoch [5/10], Loss: 976.052512755102
Epoch [6/10], Loss: 882.4413488520408
Epoch [7/10], Loss: 798.506556122449
Epoch [8/10], Loss: 717.8942155612245
Epoch [9/10], Loss: 640.0279145408164
Epoch [10/10], Loss: 553.5887069515306
Epoch [1/10], Loss: 2518.559955357143
Epoch [2/10], Loss: 2153.368125
Epoch [3/10], Loss: 1720.134980867347
Epoch [4/10], Loss: 1357.277844387755
Epoch [5/10], Loss: 1103.2917346938775
Epoch [6/10], Loss: 905.0523820153061
Epoch [7/10], Loss: 771.0109518494897
Epoch [8/10], Loss: 678.1053730867347
Epoch [9/10], Loss: 598.4911862244898
Epoch [10/10], Loss: 541.2546667729592
--- Finished SGD
--- Starting GA
--- -- Finished fitness evaluation, length: 2
--- -- Finished creating offspring population
--- -- Finished evaluating fitness of 

--- -- Finished evaluating fitness of offspring population
--- Finished GA,Time:-1025.9149074554443ms
Currently in generation 9
--- Starting SGD
Epoch [1/10], Loss: 7.570862638512436
Epoch [2/10], Loss: 6.210951345015546
Epoch [3/10], Loss: 5.713553230129943
Epoch [4/10], Loss: 5.2728635453204715
Epoch [5/10], Loss: 4.750395769391741
Epoch [6/10], Loss: 4.253474756357621
Epoch [7/10], Loss: 3.9579871165995697
Epoch [8/10], Loss: 3.544544461697948
Epoch [9/10], Loss: 3.1382706373565052
Epoch [10/10], Loss: 3.3715347476881377
Epoch [1/10], Loss: 6.68926397206832
Epoch [2/10], Loss: 6.350241562201052
Epoch [3/10], Loss: 5.6303149164939414
Epoch [4/10], Loss: 5.465822757020288
Epoch [5/10], Loss: 4.790492180026307
Epoch [6/10], Loss: 4.261333854830995
Epoch [7/10], Loss: 3.87468158332669
Epoch [8/10], Loss: 3.5695894140126754
Epoch [9/10], Loss: 3.5101175860969387
Epoch [10/10], Loss: 2.8641651636240435
--- Finished SGD
--- Starting GA
--- -- Finished fitness evaluation, length: 2
--- -- F

# Test Model

In [14]:
# Test the model
def test_function(network, data_loader):
    # init accuracy
    accuracy = 0.0
    
    network.to(device)
    network.eval()
    
    with torch.no_grad():
        correct = 0
        total = 0
        for genes, types in data_loader:
            genes = genes.to(device)
            types = types.to(device)
            outputs = network(genes)
            _, predicted = torch.max(outputs.data, 1)
            
            total += genes.size(0)
            
            correct += (predicted == types).sum().item()

        accuracy =  correct / total
        #print('Accuracy of the model on the test images: {} %'.format(accuracy))
        
    # send network back to cpu
    network.cpu()
    
    return accuracy

# Test

In [15]:
test_accuracy = [test_function(trained_population[i],test_loader) for i in range(pop_size)]

In [16]:
test_accuracy

[0.6971428571428572, 0.6971428571428572]

In [17]:
np.array(test_accuracy).mean()

0.6971428571428572